In [1]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import traceback
import cloudscraper
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
def get_cars(
  make="BMW", 
  model="5 SERIES", 
  postcode="SW1A 0AA", 
  radius=1500, 
  min_year=1995, 
  max_year=1995, 
  include_writeoff="include", 
  max_attempts_per_page=1, 
  verbose=False,
  min_price= 0,
  max_price = 99999):

    # To bypass Cloudflare protection
    scraper = cloudscraper.create_scraper()

    # Basic variables
    results = []
    n_this_year_results = 0

    url = "https://www.autotrader.co.uk/results-car-search"

    keywords = {}
    keywords["mileage"] = ["miles"]
    keywords["BHP"] = ["BHP"]
    keywords["transmission"] = ["Automatic", "Manual"]
    keywords["fuel"] = [
      "Petrol", 
      "Diesel", 
      "Electric", 
      "Hybrid – Diesel/Electric Plug-in", 
      "Hybrid – Petrol/Electric", 
      "Hybrid – Petrol/Electric Plug-in"
    ]
    keywords["owners"] = ["owners"]
    keywords["body"] = [
      "Coupe", 
      "Convertible", 
      "Estate", 
      "Hatchback", 
      "MPV", 
      "Pickup", 
      "SUV", 
      "Saloon"
    ]
    keywords["ULEZ"] = ["ULEZ"]
    keywords["year"] = [" reg)"]
    keywords["engine"] = ["engine"]

    # Set up parameters for query to autotrader.co.uk
    params = {
        "sort": "relevance",
        "postcode": postcode,
        "price-from" : min_price,
        "price-to" : max_price,
        "radius": radius,
        "make": make,
        "model": model,
        "search-results-price-type": "total-price",
        "search-results-year": "select-year",
    }

    if (include_writeoff == "include"):
        params["writeoff-categories"] = "on"
    elif (include_writeoff == "exclude"):
        params["exclude-writeoff-categories"] = "on"
    elif (include_writeoff == "writeoff-only"):
        params["only-writeoff-categories"] = "on"
        
    year = min_year
    page = 1
    attempt = 1

    try:
        while year <= max_year:
            params["year-from"] = year
            params["year-to"] = year
            params["page"] = page

            r = scraper.get(url, params=params)
            if verbose:
                print("Year:     ", year)
                print("Page:     ", page)
                print("Response: ", r)

            try:
                if r.status_code != 200:   # If not successful (e.g. due to bot protection)
                    attempt = attempt + 1  # Log as an attempt
                    if attempt <= max_attempts_per_page:
                        if verbose:
                            print("Exception. Starting attempt #", attempt, "and keeping at page #", page)
                    else:
                        page = page + 1
                        attempt = 1
                        if verbose:
                            print("Exception. All attempts exhausted for this page. Skipping to next page #", page)

                else:

                    j = r.json()
                    s = BeautifulSoup(j["html"], features="html.parser")

                    articles = s.find_all("article", attrs={"data-standout-type":""})

                    # If no results or reached end of results...
                    if len(articles) == 0 or r.url[r.url.find("page=")+5:] != str(page):
                        if verbose:
                            print("Found total", n_this_year_results, "results for year", year, "across", page-1, "pages")
                            if year+1 <= max_year:
                                print("Moving on to year", year + 1)
                                print("---------------------------------")

                        # Increment year and reset relevant variables
                        type(year)
                        year = year + 1
                        page = 1
                        attempt = 1
                        n_this_year_results = 0
                    else:
                        for article in articles:
                            car = {}
                            car["name"] = article.find("h3", {"class": "product-card-details__title"}).text.strip()             
                            car["link"] = "https://www.autotrader.co.uk" + \
                                  article.find("a", {"class": "listing-fpa-link"})["href"][: article.find("a", {"class": "listing-fpa-link"})["href"] \
                                  .find("?")]
                            car["price"] = article.find("div", {"class": "product-card-pricing__price"}).text.strip()

                            seller_info = article.find("ul", {"class": "product-card-seller-info__specs"}).text.strip()
                            car["seller"] = " ".join(seller_info.split())

                            key_specs_bs_list = article.find("ul", {"class": "listing-key-specs"}).find_all("li")
                            
                            for key_spec_bs_li in key_specs_bs_list:

                                key_spec_bs = key_spec_bs_li.text

                                if any(keyword in key_spec_bs for keyword in keywords["mileage"]):
                                    car["mileage"] = int(key_spec_bs[:key_spec_bs.find(" miles")].replace(",",""))
                                elif any(keyword in key_spec_bs for keyword in keywords["BHP"]):
                                    car["BHP"] = int(key_spec_bs[:key_spec_bs.find("BHP")])
                                elif any(keyword in key_spec_bs for keyword in keywords["transmission"]):
                                    car["transmission"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["fuel"]):
                                    car["fuel"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["owners"]):
                                    car["owners"] = int(key_spec_bs[:key_spec_bs.find(" owners")])
                                elif any(keyword in key_spec_bs for keyword in keywords["body"]):
                                    car["body"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["ULEZ"]):
                                    car["ULEZ"] = key_spec_bs
                                elif any(keyword in key_spec_bs for keyword in keywords["year"]):
                                    car["year"] = key_spec_bs
                                elif key_spec_bs[1] == "." and key_spec_bs[3] == "L":
                                    car["engine"] = key_spec_bs

                            results.append(car)
                            n_this_year_results = n_this_year_results + 1

                        page = page + 1
                        attempt = 1

                        if verbose:
                            print("Car count: ", len(results))
                            print("---------------------------------")

            except KeyboardInterrupt:
                break

            except:
                traceback.print_exc()
                attempt = attempt + 1
                if attempt <= max_attempts_per_page:
                    if verbose:
                        print("Exception. Starting attempt #", attempt, "and keeping at page #", page)
                else:
                    page = page + 1
                    attempt = 1
                    if verbose:
                        print("Exception. All attempts exhausted for this page. Skipping to next page #", page)

    except KeyboardInterrupt:
        pass

    return pd.DataFrame(results)

In [3]:
#### parameters needed #####
#location 1 and location 2 to compare
#radius for each 
#car price and comparison
#minimum and maximum purchase price 

In [30]:
#leave as blank to include all within that range 
#postcode as 10 
#loation 1 is the area to buy the car and 2 is to sell 

make_1="" 
model_1="" 
postcode_1="MK3 6JS" 
radius_1=30 
min_year_1=2005 
max_year_1=2010 
min_price_1= 0
max_price_1 = 10000

make_2="" 
model_2="" 
postcode_2="DT1 3GJ" 
radius_2=30 
min_year_2=2005 
max_year_2=2010 
min_price_2= 0
max_price_2 = 10000


location_1 = get_cars(make = make_1, 
                      model = model_1,
                      postcode = postcode_1,
                      radius = radius_1 ,
                      min_year = min_year_1 ,
                      max_year = max_year_1 ,
                      min_price = min_price_1,
                      max_price = max_price_1)

location_2 = get_cars(make = make_2, 
                      model = model_2,
                      postcode = postcode_2,
                      radius = radius_2 ,
                      min_year = min_year_2 ,
                      max_year = max_year_2 ,
                      min_price = min_price_2,
                      max_price = max_price_2)

location_1['year'] = location_1['year'].dropna().apply(lambda x: int(str(x).split('(')[0]))
location_2['year'] = location_2['year'].dropna().apply(lambda x: int(str(x).split('(')[0]))

location_1['period+1'] = location_1['year'] + 1
location_2['period+1'] = location_2['year'] + 1

def postcode_add(df,postcode):
    for i in df.columns:
        concat_col = i + ' ' + postcode
        df = df.rename({i: concat_col.replace(' ','_')}, axis=1)

    return df

location_1 = postcode_add(location_1,postcode_1)

location_2 = postcode_add(location_2,postcode_2)
 

In [44]:
#give a take a year for model to model.
#minimal mileage differen between both.
#find the price difference 

In [35]:
#column chooser 
def col_select(df,column):
    final_cols = []
    for i in df.columns:
        for j in column:
            if j in i:
                final_cols.append(i)
    
    print(final_cols)

col_select(location_1,['period+1','year'])

name_MK3_6JS
link_MK3_6JS
price_MK3_6JS
seller_MK3_6JS
year_MK3_6JS
body_MK3_6JS
mileage_MK3_6JS
engine_MK3_6JS
BHP_MK3_6JS
transmission_MK3_6JS
fuel_MK3_6JS
owners_MK3_6JS
period+1_MK3_6JS
['year_MK3_6JS', 'period+1_MK3_6JS']


In [103]:
df1 = location_1.merge(location_2, how = 'inner', left_on = ['name','year'] ,right_on = ['name','year'],suffixes = ('_1','_2'))
df1['year_1'] = df1['year']
df1['year_2'] = df1['year']
df2 = location_1.merge(location_2 ,how = 'inner', left_on = ['name','year'] ,right_on = ['name','year + 1'],suffixes = ('_1','_2'))
df3 = location_1.merge(location_2 ,how = 'inner', right_on = ['name','year'] ,left_on = ['name','year + 1'],suffixes = ('_1','_2'))

finalcars_pre = pd.concat([df1,df2,df3],axis = 0)[['name','link_1','price_1','price_2','year_1','year_2','mileage_1','mileage_2','body_1','body_2','engine_1','engine_2','body_1','body_2','postcode_1','postcode_2']]
finalcars = finalcars_pre[finalcars_pre['postcode_1'] != finalcars_pre['postcode_2']]


In [113]:
finalcars[finalcars['name'] == 'Renault Megane']

,name,link_1,price_1,price_2,year_1,year_2,mileage_1,mileage_2,body_1,body_2,engine_1,engine_2,body_1,body_2,postcode_1,postcode_2
212,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,490","£1,990",2005.0,2005.0,67000,89000.0,Hatchback,Hatchback,1.6L,1.6L,Hatchback,Hatchback,MK3 6JS,DT1 3GJ
213,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,490","£2,490",2005.0,2005.0,67000,67000.0,Hatchback,Hatchback,1.6L,1.6L,Hatchback,Hatchback,MK3 6JS,DT1 3GJ
227,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,250","£3,790",2006.0,2006.0,105000,61000.0,Hatchback,Estate,2.0L,1.6L,Hatchback,Estate,MK3 6JS,DT1 3GJ
228,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,£995,"£3,790",2006.0,2006.0,120075,61000.0,Hatchback,Estate,1.5L,1.6L,Hatchback,Estate,MK3 6JS,DT1 3GJ
229,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£1,490","£3,790",2006.0,2006.0,130900,61000.0,Hatchback,Estate,1.5L,1.6L,Hatchback,Estate,MK3 6JS,DT1 3GJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£6,990","£3,750",2008.0,2009.0,89000,47500.0,Hatchback,Hatchback,2.0L,1.5L,Hatchback,Hatchback,MK3 6JS,DT1 3GJ
2205,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£6,990","£3,395",2008.0,2009.0,89000,54300.0,Hatchback,Estate,2.0L,1.6L,Hatchback,Estate,MK3 6JS,DT1 3GJ
3076,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,499","£2,295",2009.0,2010.0,70030,110000.0,Hatchback,Hatchback,1.6L,1.6L,Hatchback,Hatchback,MK3 6JS,DT1 3GJ
3077,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£1,845","£2,295",2009.0,2010.0,125000,110000.0,Coupe,Hatchback,1.6L,1.6L,Coupe,Hatchback,MK3 6JS,DT1 3GJ


In [112]:
df2[df2['name'] == 'Renault Megane']

,name,link_1,price_1,seller_1,year_1,body_1,mileage_1,engine_1,BHP_1,transmission_1,...,year_2,body_2,mileage_2,engine_2,BHP_2,transmission_2,fuel_2,owners_2,year + 1_2,postcode_2
0,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,250",welwyn (26 miles),2006.0,Hatchback,105000,2.0L,225.0,Manual,...,2005,Hatchback,89000.0,1.6L,115.0,Manual,Petrol,NaN,2006,DT1 3GJ
1,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,250",welwyn (26 miles),2006.0,Hatchback,105000,2.0L,225.0,Manual,...,2005,Hatchback,67000.0,1.6L,111.0,Manual,Petrol,NaN,2006,DT1 3GJ
2,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,£995,rushden (21 miles),2006.0,Hatchback,120075,1.5L,85.0,Manual,...,2005,Hatchback,89000.0,1.6L,115.0,Manual,Petrol,NaN,2006,DT1 3GJ
3,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,£995,rushden (21 miles),2006.0,Hatchback,120075,1.5L,85.0,Manual,...,2005,Hatchback,67000.0,1.6L,111.0,Manual,Petrol,NaN,2006,DT1 3GJ
4,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£1,490",4.5 (8 reviews) sandy (22 miles),2006.0,Hatchback,130900,1.5L,106.0,Manual,...,2005,Hatchback,89000.0,1.6L,115.0,Manual,Petrol,NaN,2006,DT1 3GJ
5,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£1,490",4.5 (8 reviews) sandy (22 miles),2006.0,Hatchback,130900,1.5L,106.0,Manual,...,2005,Hatchback,67000.0,1.6L,111.0,Manual,Petrol,NaN,2006,DT1 3GJ
6,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£1,995",4.9 (92 reviews) milton keynes (3 miles),2006.0,Hatchback,113567,1.9L,130.0,Manual,...,2005,Hatchback,89000.0,1.6L,115.0,Manual,Petrol,NaN,2006,DT1 3GJ
7,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£1,995",4.9 (92 reviews) milton keynes (3 miles),2006.0,Hatchback,113567,1.9L,130.0,Manual,...,2005,Hatchback,67000.0,1.6L,111.0,Manual,Petrol,NaN,2006,DT1 3GJ
8,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,495",4.8 (136 reviews) st. albans (26 miles),2006.0,Hatchback,57600,1.6L,111.0,Automatic,...,2005,Hatchback,89000.0,1.6L,115.0,Manual,Petrol,NaN,2006,DT1 3GJ
9,Renault Megane,https://www.autotrader.co.uk/car-details/20230...,"£2,495",4.8 (136 reviews) st. albans (26 miles),2006.0,Hatchback,57600,1.6L,111.0,Automatic,...,2005,Hatchback,67000.0,1.6L,111.0,Manual,Petrol,NaN,2006,DT1 3GJ
